In [ ]:
import arxiv
import requests
import json
import os
import tarfile
import gzip
import shutil
import time
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import logging
from datetime import datetime
import re
import psutil  # For memory tracking
import sys

In [ ]:
# Configuration
class Config:
    START_MONTH = "2023-09"
    START_ID = "2848" # 2848
    END_MONTH = "2023-09" 
    END_ID = "2850"  # 7847

    BASE_DIR = "./data"
    STUDENT_ID = "23127388"
    RATE_LIMIT_SEMANTIC_SCHOLAR = 3.0  # seconds between requests (increased from 1.1)
    RATE_LIMIT_ARXIV = 3.0  # seconds between arXiv requests
    MAX_RETRIES = 3
    RETRY_DELAY = 60  # seconds to wait when hitting rate limit (429)


def generate_arxiv_ids():
    """Generate list of arXiv IDs based on the given range"""
    start_num = int(Config.START_ID)
    end_num = int(Config.END_ID)
    
    # Convert "2023-09" to "2309" format for arXiv API
    year_month = Config.START_MONTH.replace("-", "")[2:]  # "2023-09" -> "2309"
    
    arxiv_ids = []
    for i in range(start_num, end_num + 1):
        arxiv_id = f"{year_month}.{i:05d}"
        arxiv_ids.append(arxiv_id)
    
    logging.info(f"Generated {len(arxiv_ids)} arXiv IDs from {arxiv_ids[0]} to {arxiv_ids[-1]}")
    return arxiv_ids

def format_arxiv_id_for_path(arxiv_id):
    """Convert arXiv ID to folder name format (yyyymm-id)"""
    
    # Handle old format: "hep-ph/0512066" -> "hep-ph-0512066"
    if '/' in arxiv_id:
        return arxiv_id.replace('/', '-')
    
    # Handle new format: "2309.02848" -> "2309-02848"
    if '.' in arxiv_id:
        return arxiv_id.replace('.', '-')
    
    return arxiv_id

In [ ]:
def get_paper_versions(arxiv_id):
    """Get all versions of a paper"""
    try:
        # Rate limiting for arXiv API
        time.sleep(Config.RATE_LIMIT_ARXIV)
        
        client = arxiv.Client()
        search = arxiv.Search(id_list=[arxiv_id])
        papers = list(client.results(search))
        
        if not papers:
            return []
        
        paper = papers[0]
        
        # Try to get version count from the paper object
        # arxiv.py doesn't directly expose version info, so we'll try the entry_id
        versions = []
        
        # Get the latest version number from the entry_id
        # Format: http://arxiv.org/abs/2309.02848v2
        if hasattr(paper, 'entry_id') and paper.entry_id:
            match = re.search(r'v(\d+)$', paper.entry_id)
            if match:
                latest_version = int(match.group(1))
                for v in range(1, latest_version + 1):
                    versions.append(f"{arxiv_id}v{v}")
            else:
                versions.append(f"{arxiv_id}v1")
        else:
            versions.append(f"{arxiv_id}v1")
        
        return versions
        
    except Exception as e:
        logging.error(f"Error getting versions for {arxiv_id}: {str(e)}")
        return [f"{arxiv_id}v1"]  # At least try version 1

def download_arxiv_paper(arxiv_id_with_version, save_dir):
    """Download arXiv paper source and metadata for a specific version"""
    try:
        # Rate limiting for arXiv API
        time.sleep(Config.RATE_LIMIT_ARXIV)
        
        # Extract base ID without version
        base_id = re.sub(r'v\d+$', '', arxiv_id_with_version)
        
        client = arxiv.Client()
        search = arxiv.Search(id_list=[base_id])
        papers = list(client.results(search))
        
        if not papers:
            logging.warning(f"Paper not found: {arxiv_id_with_version}")
            return None, None
        
        paper = papers[0]
        
        # Download source file
        source_filename = f"{arxiv_id_with_version}.tar.gz"
        source_path = os.path.join(save_dir, source_filename)
        
        try:
            paper.download_source(dirpath=save_dir, filename=source_filename)
        except Exception as e:
            logging.warning(f"Could not download source for {arxiv_id_with_version}: {str(e)}")
            return None, None
        
        # Extract metadata (same for all versions)
        metadata = {
            "title": paper.title,
            "authors": [author.name for author in paper.authors],
            "submission_date": paper.published.isoformat() if paper.published else None,
            "revised_dates": [paper.updated.isoformat()] if paper.updated else [],
            "publication_venue": paper.journal_ref if hasattr(paper, 'journal_ref') else None,
            "abstract": paper.summary,
            "categories": paper.categories,
            "arxiv_id": base_id
        }
        
        logging.info(f"✓ Downloaded {arxiv_id_with_version}: {paper.title[:50]}...")
        return metadata, source_path
        
    except Exception as e:
        logging.error(f"✗ Error downloading {arxiv_id_with_version}: {str(e)}")
        return None, None

def extract_and_remove_figures(tar_path, extract_dir):
    """Extract tar.gz/gz, keep only .tex files and empty folders"""
    import gzip
    import shutil
    
    try:
        # Create extraction directory
        os.makedirs(extract_dir, exist_ok=True)
        
        # Try to extract as tar.gz first
        try:
            with tarfile.open(tar_path, 'r:gz') as tar:
                tar.extractall(path=extract_dir)
            logging.info(f"Extracted tar.gz archive to {extract_dir}")
        except tarfile.ReadError:
            # Not a tar archive, might be a single .gz file
            logging.warning(f"{tar_path} is not a tar archive, trying as single .gz file")
            try:
                # Extract as single gzip file
                output_file = os.path.join(extract_dir, "main.tex")
                with gzip.open(tar_path, 'rb') as f_in:
                    with open(output_file, 'wb') as f_out:
                        shutil.copyfileobj(f_in, f_out)
                logging.info(f"Extracted single .gz file to {output_file}")
            except Exception as gz_error:
                logging.error(f"Failed to extract as .gz: {str(gz_error)}")
                # Try copying as plain text (some papers might not have source)
                try:
                    output_file = os.path.join(extract_dir, "README.txt")
                    with open(output_file, 'w') as f:
                        f.write("Source not available or extraction failed.\n")
                        f.write(f"Original file: {tar_path}\n")
                    logging.warning(f"Could not extract source, created placeholder")
                except:
                    return True
        
        # Remove ALL non-.tex files but KEEP folders (including Figures/, vis/, etc.)
        # Only keep .tex files and empty folders
        print(f"🧹 Cleaning up non-.tex files in {os.path.basename(extract_dir)}...")
        removed_count = 0
        failed_count = 0
        
        for root, dirs, files in os.walk(extract_dir):
            for file in files:
                file_ext = os.path.splitext(file)[1].lower()
                # Keep ONLY .tex files, remove everything else
                if file_ext != '.tex':
                    file_path = os.path.join(root, file)
                    try:
                        os.remove(file_path)
                        removed_count += 1
                    except Exception as e:
                        failed_count += 1
                        logging.warning(f"Failed to remove {file_path}: {str(e)}")
        
        logging.info(f"Cleanup: Removed {removed_count} non-.tex files, {failed_count} failed from {extract_dir}")
        
        return True
        
    except Exception as e:
        logging.error(f"Error extracting {tar_path}: {str(e)}")
        return False



In [ ]:
def get_references_from_semantic_scholar(arxiv_id):
    """Get references from Semantic Scholar API with rate limiting and retry"""
    
    for attempt in range(Config.MAX_RETRIES):
        time.sleep(Config.RATE_LIMIT_SEMANTIC_SCHOLAR)  # Rate limiting
        
        try:
            url = f"https://api.semanticscholar.org/graph/v1/paper/arXiv:{arxiv_id}"
            params = {
                "fields": "references,references.externalIds,references.title,references.authors,references.year,references.publicationVenue"
            }
            
            response = requests.get(url, params=params)
            
            if response.status_code == 200:
                data = response.json()
                references_data = {}
                
                for ref in data.get("references", []):
                    # Safety check: ref might be None
                    if not ref or not isinstance(ref, dict):
                        continue
                    
                    external_ids = ref.get("externalIds")
                    if not external_ids or not isinstance(external_ids, dict):
                        continue
                    
                    arxiv_id_ref = external_ids.get("ArXiv")
                    if not arxiv_id_ref:
                        continue
                    
                    try:
                        # Validate arXiv ID format before processing
                        # ONLY accept new format: "2309.02848" or "2309.02848v1"
                        # SKIP old format: "hep-ph/0512066", "cs/0001001"
                        
                        # Remove version suffix if present
                        clean_id = re.sub(r'v\d+$', '', str(arxiv_id_ref))
                        
                        # Check if it's NEW format ONLY (must have dot, NOT slash)
                        if '.' not in clean_id or '/' in clean_id:
                            logging.debug(f"Skipping non-new-format arXiv ID: {arxiv_id_ref}")
                            continue
                        
                        # Validate new format pattern: YYMM.NNNNN
                        if not re.match(r'^\d{4}\.\d+$', clean_id):
                            logging.warning(f"Skipping invalid new-format arXiv ID: {arxiv_id_ref}")
                            continue
                        
                        # Format arXiv ID for path
                        formatted_id = format_arxiv_id_for_path(clean_id)
                        
                        # Safety check for authors
                        authors_list = ref.get("authors", [])
                        if authors_list is None:
                            authors_list = []
                        
                        ref_metadata = {
                            "title": ref.get("title", ""),
                            "authors": [author.get("name", "") for author in authors_list if author and isinstance(author, dict)],
                            "submission_date": f"{ref.get('year', '')}-01-01" if ref.get("year") else None,
                            "revised_dates": [],  # Semantic Scholar doesn't provide revised dates
                            "publication_venue": ref.get("publicationVenue", ""),
                            "year": ref.get("year")
                        }
                        
                        references_data[formatted_id] = ref_metadata
                        
                    except Exception as parse_error:
                        logging.warning(f"Error parsing reference {arxiv_id_ref}: {str(parse_error)}")
                        continue
                
                logging.info(f"Found {len(references_data)} arXiv references for {arxiv_id}")
                return references_data
                
            elif response.status_code == 404:
                logging.warning(f"Paper {arxiv_id} not found in Semantic Scholar")
                return {}
                
            elif response.status_code == 429:
                # Rate limit exceeded - wait longer and retry
                wait_time = Config.RETRY_DELAY * (attempt + 1)
                logging.warning(f"Rate limit hit for {arxiv_id} (attempt {attempt + 1}/{Config.MAX_RETRIES}). Waiting {wait_time}s...")
                if attempt < Config.MAX_RETRIES - 1:
                    time.sleep(wait_time)
                    continue
                else:
                    logging.error(f"Failed to get references for {arxiv_id} after {Config.MAX_RETRIES} attempts")
                    return {}
            else:
                logging.warning(f"Semantic Scholar API error for {arxiv_id}: {response.status_code}")
                return {}
                
        except Exception as e:
            logging.error(f"Error getting references for {arxiv_id}: {str(e)}")
            if attempt < Config.MAX_RETRIES - 1:
                time.sleep(Config.RETRY_DELAY)
                continue
            return {}
    
    return {}

In [ ]:
def process_single_paper(arxiv_id, base_dir):
    """Process a single arXiv paper - download all versions, extract, get references"""
    try:
        # Create paper directory
        paper_dir_name = format_arxiv_id_for_path(arxiv_id)
        paper_dir = os.path.join(base_dir, Config.STUDENT_ID, paper_dir_name)
        tex_dir = os.path.join(paper_dir, "tex")
        os.makedirs(tex_dir, exist_ok=True)
        
        # Step 1: Get all versions of the paper
        versions = get_paper_versions(arxiv_id)
        logging.info(f"Found {len(versions)} version(s) for {arxiv_id}")
        
        if not versions:
            logging.warning(f"No versions found for {arxiv_id}")
            return False
        
        # Step 2: Download all versions
        metadata = None
        successful_downloads = 0
        
        for version in versions:
            version_metadata, source_path = download_arxiv_paper(version, paper_dir)
            
            if version_metadata and source_path:
                # Store metadata from first successful download
                if metadata is None:
                    metadata = version_metadata
                
                # Version folder name: "2309-02848v1" instead of "v1"
                version_formatted = format_arxiv_id_for_path(version)
                
                # Extract to version-specific subdirectory in tex/
                # Only keep .tex files and empty folders
                version_tex_dir = os.path.join(tex_dir, version_formatted)
                if extract_and_remove_figures(source_path, version_tex_dir):
                    successful_downloads += 1
                
                # Clean up: remove original tar.gz file
                if os.path.exists(source_path):
                    os.remove(source_path)
        
        if successful_downloads == 0:
            logging.error(f"Failed to download any version for {arxiv_id}")
            return False
        
        # Step 3: Get references from Semantic Scholar (only once for the paper)
        # Skip references without arXiv ID (already filtered in get_references_from_semantic_scholar)
        references = get_references_from_semantic_scholar(arxiv_id)
        
        # Step 4: Save all data
        # Save metadata
        with open(os.path.join(paper_dir, "metadata.json"), 'w', encoding='utf-8') as f:
            json.dump(metadata, f, indent=2, ensure_ascii=False)
        
        # Save references (only those with arXiv IDs)
        with open(os.path.join(paper_dir, "references.json"), 'w', encoding='utf-8') as f:
            json.dump(references, f, indent=2, ensure_ascii=False)
        
        logging.info(f"✓ Successfully processed {arxiv_id} ({successful_downloads}/{len(versions)} versions)")
        return True
        
    except Exception as e:
        logging.error(f"✗ Failed to process {arxiv_id}: {str(e)}")
        return False

def scrape_arxiv_papers():
    """Main function to scrape all arXiv papers in the given range"""
    # Generate arXiv IDs
    arxiv_ids = generate_arxiv_ids()
    
    # Create base directory
    base_dir = Path(Config.BASE_DIR)
    base_dir.mkdir(exist_ok=True)
    
    # Statistics
    stats = {
        'total': len(arxiv_ids),
        'success': 0,
        'failed': 0,
        'failed_ids': [],
        'start_time': datetime.now()
    }
    
    logging.info(f"Starting to scrape {stats['total']} papers...")
    
    # Process each paper
    for i, arxiv_id in enumerate(tqdm(arxiv_ids, desc="Scraping papers")):
        logging.info(f"Processing {arxiv_id} ({i+1}/{stats['total']})")
        
        success = process_single_paper(arxiv_id, base_dir)
        
        if success:
            stats['success'] += 1
        else:
            stats['failed'] += 1
            stats['failed_ids'].append(arxiv_id)
        
        # Progress update every 10 papers
        if (i + 1) % 10 == 0:
            logging.info(f"Progress: {i+1}/{stats['total']} - Success: {stats['success']}, Failed: {stats['failed']}")
    
    # Calculate final statistics
    stats['end_time'] = datetime.now()
    stats['duration'] = stats['end_time'] - stats['start_time']
    stats['success_rate'] = (stats['success'] / stats['total']) * 100
    
    return stats

In [ ]:
# Block 6:

def calculate_statistics():
    """Calculate statistics about scraped data"""
    data_dir = Path(Config.BASE_DIR) / Config.STUDENT_ID
    
    if not data_dir.exists():
        return {"error": "Data directory not found"}
    
    stats = {
        'total_papers': 0,
        'paper_sizes': [],
        'reference_counts': [],
        'successful_references': 0,
        'total_references': 0
    }
    
    for paper_dir in data_dir.iterdir():
        if paper_dir.is_dir():
            stats['total_papers'] += 1
            
            # Calculate paper size
            tex_dir = paper_dir / "tex"
            if tex_dir.exists():
                paper_size = sum(f.stat().st_size for f in tex_dir.rglob('*') if f.is_file())
                stats['paper_sizes'].append(paper_size)
            
            # Count references
            references_file = paper_dir / "references.json"
            if references_file.exists():
                with open(references_file, 'r') as f:
                    references = json.load(f)
                    ref_count = len(references)
                    stats['reference_counts'].append(ref_count)
                    stats['total_references'] += ref_count
                    stats['successful_references'] += sum(1 for ref in references.values() if ref.get('title'))
    
    # Calculate averages
    if stats['paper_sizes']:
        stats['avg_paper_size_kb'] = sum(stats['paper_sizes']) / len(stats['paper_sizes']) / 1024
        stats['max_paper_size_kb'] = max(stats['paper_sizes']) / 1024 if stats['paper_sizes'] else 0
        stats['min_paper_size_kb'] = min(stats['paper_sizes']) / 1024 if stats['paper_sizes'] else 0
    
    if stats['reference_counts']:
        stats['avg_references_per_paper'] = sum(stats['reference_counts']) / len(stats['reference_counts'])
        stats['reference_success_rate'] = (stats['successful_references'] / stats['total_references'] * 100) if stats['total_references'] > 0 else 0
    
    return stats

def generate_report(stats, scraping_stats):
    """Generate comprehensive report (in-memory only, not saved to file)"""
    report = {
        "scraping_summary": {
            "student_id": Config.STUDENT_ID,
            "arxiv_id_range": f"{Config.START_MONTH}.{Config.START_ID} to {Config.END_MONTH}.{Config.END_ID}",
            "total_papers_attempted": scraping_stats['total'],
            "successfully_scraped": scraping_stats['success'],
            "failed": scraping_stats['failed'],
            "success_rate": f"{scraping_stats['success_rate']:.2f}%",
            "duration": str(scraping_stats['duration'])
        },
        "data_statistics": stats,
        "performance_metrics": {
            "tools_used": ["arxiv.py", "requests", "Semantic Scholar API"],
            "rate_limiting": f"{Config.RATE_LIMIT_SEMANTIC_SCHOLAR}s between Semantic Scholar requests"
        }
    }
    
    # No longer save report to file
    return report

In [ ]:
# Block 7

# def main():
#     """Main execution function"""
#     separator = "-------" * 10
    
#     print(separator)
#     print("🚀 Starting arXiv Paper Scraper")
#     print(f"📊 Range: {Config.START_MONTH}.{Config.START_ID} to {Config.END_MONTH}.{Config.END_ID}")
#     print(f"🎯 Student ID: {Config.STUDENT_ID}")
#     print(separator)
    
#     # Start scraping
#     scraping_stats = scrape_arxiv_papers()
    
#     print("\n" + separator)
#     print("📈 Scraping Completed! Calculating statistics...")
#     print(separator)
    
#     # Calculate statistics
#     stats = calculate_statistics()
    
#     # Generate report
#     report = generate_report(stats, scraping_stats)
    
#     # Print summary
#     print("\n" + separator)
#     print("🎉 FINAL SUMMARY:")
#     print(separator)
#     print(f"Total papers attempted: {report['scraping_summary']['total_papers_attempted']}")
#     print(f"Successfully scraped: {report['scraping_summary']['successfully_scraped']}")
#     print(f"Failed: {report['scraping_summary']['failed']}")
#     print(f"Success rate: {report['scraping_summary']['success_rate']}")
#     print(f"Duration: {report['scraping_summary']['duration']}")
#     print(separator)
    
#     if 'avg_paper_size_kb' in stats:
#         print(f"Average paper size: {stats['avg_paper_size_kb']:.2f} KB")
#         print(f"Max paper size: {stats['max_paper_size_kb']:.2f} KB")
#         print(f"Min paper size: {stats['min_paper_size_kb']:.2f} KB")
#     if 'avg_references_per_paper' in stats:
#         print(f"Average references per paper: {stats['avg_references_per_paper']:.2f}")
#         print(f"Reference scraping success rate: {stats.get('reference_success_rate', 0):.2f}%")
    
#     print(separator)
#     print(f"📁 Data saved in: {Config.BASE_DIR}/{Config.STUDENT_ID}/")
#     print(separator)
    
#     # Print failed IDs if any
#     if scraping_stats.get('failed_ids'):
#         print("\n⚠️  Failed paper IDs:")
#         for failed_id in scraping_stats['failed_ids']:
#             print(f"  - {failed_id}")
#         print(separator)

# # main function to run program
# if __name__ == "__main__": 
#     main()

In [ ]:
# ENHANCED METRICS COLLECTION

def get_directory_size(path):
    """Calculate total size of directory in bytes"""
    total = 0
    try:
        for entry in os.scandir(path):
            if entry.is_file():
                total += entry.stat().st_size
            elif entry.is_dir():
                total += get_directory_size(entry.path)
    except:
        pass
    return total

def process_single_paper_with_metrics(arxiv_id, base_dir, global_stats):
    """Process a single paper with detailed metrics tracking"""
    paper_start_time = time.time()
    
    try:
        # Create paper directory
        paper_dir_name = format_arxiv_id_for_path(arxiv_id)
        paper_dir = os.path.join(base_dir, Config.STUDENT_ID, paper_dir_name)
        tex_dir = os.path.join(paper_dir, "tex")
        os.makedirs(tex_dir, exist_ok=True)
        
        # Step 1: Get all versions
        versions = get_paper_versions(arxiv_id)
        if not versions:
            return False, 0, 0, 0, 0
        
        # Step 2: Download and extract all versions
        metadata = None
        successful_downloads = 0
        total_size_before = 0
        total_size_after = 0
        
        for version in versions:
            version_metadata, source_path = download_arxiv_paper(version, paper_dir)
            
            if version_metadata and source_path:
                if metadata is None:
                    metadata = version_metadata
                
                # Measure size before extraction
                if os.path.exists(source_path):
                    size_before_extract = os.path.getsize(source_path)
                else:
                    size_before_extract = 0
                
                # Extract
                version_formatted = format_arxiv_id_for_path(version)
                version_tex_dir = os.path.join(tex_dir, version_formatted)
                
                # Calculate size before cleanup
                if extract_and_remove_figures(source_path, version_tex_dir):
                    # Size after cleanup
                    size_after = get_directory_size(version_tex_dir)
                    total_size_before += size_before_extract
                    total_size_after += size_after
                    successful_downloads += 1
                
                # Clean up tar.gz
                if os.path.exists(source_path):
                    os.remove(source_path)
        
        if successful_downloads == 0:
            return False, 0, 0, 0, 0
        
        # Step 3: Get references
        references = get_references_from_semantic_scholar(arxiv_id)
        
        # Step 4: Save data
        with open(os.path.join(paper_dir, "metadata.json"), 'w', encoding='utf-8') as f:
            json.dump(metadata, f, indent=2, ensure_ascii=False)
        
        with open(os.path.join(paper_dir, "references.json"), 'w', encoding='utf-8') as f:
            json.dump(references, f, indent=2, ensure_ascii=False)
        
        paper_time = time.time() - paper_start_time
        
        return True, paper_time, total_size_before, total_size_after, len(references)
        
    except Exception as e:
        logging.error(f"Failed to process {arxiv_id}: {str(e)}")
        return False, 0, 0, 0, 0

def scrape_with_enhanced_metrics():
    """Scrape with comprehensive metrics collection"""
    # Start metrics
    overall_start_time = time.time()
    process = psutil.Process()
    initial_memory = process.memory_info().rss / 1024 / 1024  # MB
    
    # Entry discovery timing
    entry_start = time.time()
    arxiv_ids = generate_arxiv_ids()
    entry_discovery_time = time.time() - entry_start
    
    # Create base directory
    base_dir = Path(Config.BASE_DIR)
    base_dir.mkdir(exist_ok=True)
    
    # Statistics
    stats = {
        'total': len(arxiv_ids),
        'success': 0,
        'failed': 0,
        'failed_ids': [],
        'paper_times': [],
        'sizes_before': [],
        'sizes_after': [],
        'reference_counts': [],
        'memory_samples': [],
        'disk_usage_samples': []
    }
    
    # Initial disk usage
    try:
        disk_usage = psutil.disk_usage(str(base_dir))
        stats['initial_disk_free'] = disk_usage.free / 1024 / 1024 / 1024  # GB
    except:
        stats['initial_disk_free'] = 0
    
    logging.info(f"Starting to scrape {stats['total']} papers...")
    print(f"Entry discovery took {entry_discovery_time:.2f} seconds")
    
    # Process each paper
    for i, arxiv_id in enumerate(tqdm(arxiv_ids, desc="Scraping papers")):
        # Memory sampling (every 10 papers)
        if i % 10 == 0:
            current_memory = process.memory_info().rss / 1024 / 1024  # MB
            stats['memory_samples'].append(current_memory)
            
            # Disk usage
            try:
                data_dir = base_dir / Config.STUDENT_ID
                if data_dir.exists():
                    disk_used = get_directory_size(str(data_dir)) / 1024 / 1024  # MB
                    stats['disk_usage_samples'].append(disk_used)
            except:
                pass
        
        success, paper_time, size_before, size_after, ref_count = process_single_paper_with_metrics(
            arxiv_id, base_dir, stats
        )
        
        if success:
            stats['success'] += 1
            stats['paper_times'].append(paper_time)
            stats['sizes_before'].append(size_before)
            stats['sizes_after'].append(size_after)
            stats['reference_counts'].append(ref_count)
        else:
            stats['failed'] += 1
            stats['failed_ids'].append(arxiv_id)
        
        if (i + 1) % 10 == 0:
            logging.info(f"Progress: {i+1}/{stats['total']} - Success: {stats['success']}, Failed: {stats['failed']}")
    
    # Final metrics
    overall_duration = time.time() - overall_start_time
    final_memory = process.memory_info().rss / 1024 / 1024  # MB
    
    # Final disk usage
    try:
        data_dir = base_dir / Config.STUDENT_ID
        if data_dir.exists():
            final_disk_used = get_directory_size(str(data_dir)) / 1024 / 1024  # MB
        else:
            final_disk_used = 0
    except:
        final_disk_used = 0
    
    # Calculate statistics
    enhanced_stats = {
        'scraping_summary': {
            'student_id': Config.STUDENT_ID,
            'arxiv_id_range': f"{Config.START_MONTH}.{Config.START_ID} to {Config.END_MONTH}.{Config.END_ID}",
            'total_papers_attempted': stats['total'],
            'successfully_scraped': stats['success'],
            'failed': stats['failed'],
            'success_rate_percent': (stats['success'] / stats['total']) * 100 if stats['total'] > 0 else 0,
            'failed_paper_ids': stats['failed_ids']
        },
        'timing_metrics': {
            'total_duration_seconds': overall_duration,
            'total_duration_formatted': str(datetime.now() - datetime.fromtimestamp(time.time() - overall_duration)).split('.')[0],
            'entry_discovery_time_seconds': entry_discovery_time,
            'average_time_per_paper_seconds': sum(stats['paper_times']) / len(stats['paper_times']) if stats['paper_times'] else 0,
            'min_time_per_paper_seconds': min(stats['paper_times']) if stats['paper_times'] else 0,
            'max_time_per_paper_seconds': max(stats['paper_times']) if stats['paper_times'] else 0,
        },
        'storage_metrics': {
            'average_paper_size_before_cleanup_mb': sum(stats['sizes_before']) / len(stats['sizes_before']) / 1024 / 1024 if stats['sizes_before'] else 0,
            'average_paper_size_after_cleanup_mb': sum(stats['sizes_after']) / len(stats['sizes_after']) / 1024 / 1024 if stats['sizes_after'] else 0,
            'total_size_before_cleanup_mb': sum(stats['sizes_before']) / 1024 / 1024 if stats['sizes_before'] else 0,
            'total_size_after_cleanup_mb': sum(stats['sizes_after']) / 1024 / 1024 if stats['sizes_after'] else 0,
            'storage_saved_percent': ((sum(stats['sizes_before']) - sum(stats['sizes_after'])) / sum(stats['sizes_before']) * 100) if sum(stats['sizes_before']) > 0 else 0,
            'final_output_size_mb': final_disk_used,
            'final_output_size_gb': final_disk_used / 1024,
        },
        'memory_metrics': {
            'initial_memory_mb': initial_memory,
            'final_memory_mb': final_memory,
            'max_memory_used_mb': max(stats['memory_samples']) if stats['memory_samples'] else final_memory,
            'average_memory_mb': sum(stats['memory_samples']) / len(stats['memory_samples']) if stats['memory_samples'] else final_memory,
            'memory_increase_mb': final_memory - initial_memory,
        },
        'reference_metrics': {
            'total_references_collected': sum(stats['reference_counts']),
            'average_references_per_paper': sum(stats['reference_counts']) / len(stats['reference_counts']) if stats['reference_counts'] else 0,
            'min_references_per_paper': min(stats['reference_counts']) if stats['reference_counts'] else 0,
            'max_references_per_paper': max(stats['reference_counts']) if stats['reference_counts'] else 0,
        },
        'tools_used': [
            'arxiv.py - For paper download via arXiv API',
            'Semantic Scholar API - For reference extraction',
            'psutil - For memory and disk usage tracking'
        ]
    }
    
    # Save enhanced report to file
    report_file = base_dir / f"{Config.STUDENT_ID}_detailed_report.json"
    with open(report_file, 'w', encoding='utf-8') as f:
        json.dump(enhanced_stats, f, indent=2, ensure_ascii=False)
    
    logging.info(f"Enhanced report saved to {report_file}")
    
    return enhanced_stats

# Run with enhanced metrics (uncomment to use)
enhanced_report = scrape_with_enhanced_metrics()
